In [1]:
import psycopg2
import clickhouse_connect
import pandas as pd
import os
import sys


# Подключение к PostgreSQL
def conn_pg():
    try:
        conn = psycopg2.connect(
            host="localhost",
            database="dbpg",
            user="user",
            password="password",
            port=5432
        )
        # print("Connected to PostgreSQL")
        # cur = conn.cursor()
        # cur.execute(query)
        # output = cur.fetchall()
        # print("PostgreSQL version:", db_version)
        # if output:
        #     return
        # else:
        #     print("No output from the query")
        return conn    
    except Exception as e:
        print(f"Error connecting to PostgreSQL: {e}")
    
def conn_pg_close(conn):
    if conn:
        cur = conn.cursor()
        cur.close()
        conn.close()
        # print("PostgreSQL connection closed")


# Подключение к ClickHouse
def conn_ch():
    try:
        client = clickhouse_connect.get_client(
            host='localhost',
            username='user',
            password='password',
            database='dbch',
            port=8123
        )
        # print("Connected to ClickHouse")
        # version = client.command('SELECT version()')
        # print("ClickHouse version:", version)
        return client
    except Exception as e:
        print(f"Error connecting to ClickHouse: {e}")

        # print("Connected to ClickHouse")
        # version = client.command('SELECT version()')

        # print("ClickHouse version:", version)


# import tools
# Get the parent directory
parent_dir = os.path.abspath(os.path.join(os.getcwd(), "../..")) # for subfolder

# find tools in parent dir
if os.path.isdir(os.path.join(parent_dir, 'tools')):
    # Add parent directory to sys.path if found
    sys.path.append(parent_dir)
    
else:
    # for run in spark
    parent_dir = os.path.abspath(os.path.join(os.getcwd(), "../airflow/airflow_data"))
    
    # Add parent directory to sys.path
    sys.path.append(parent_dir)


from tools import pd_tools
from tools.paths import Paths
from tools.db_tools import DbTools


# import paths
root_path = '..' # for local folder
paths = Paths(root_path)
data_path = paths.data_path
tmp_path = paths.tmp_path
prod_data_path = paths.prod_data_path
dev_data_path = paths.dev_data_path
prod_db = paths.prod_db
dev_db = paths.dev_db
tmp_db = paths.tmp_db
tmp_data_path = paths.tmp_data_path

# db_tools = DbTools(data_path, tmp_path, client)

## Load data and analyze it

In [2]:
raw_data = pd.read_csv(f'{data_path}/data-9-structure-4.csv')
raw_data.head()

C:\Users\pmz\AppData\Local\Temp\ipykernel_15748\3635358273.py:1: DtypeWarning: Columns (2,12,13,14,17,19,20,32,35,36,37,40,44) have mixed types. Specify dtype option on import or set low_memory=False.
  raw_data = pd.read_csv(f'{data_path}/data-9-structure-4.csv')


,Название фильма,Hаименование на иностранном языке,Номер удостоверения,Дата регистрации удостоверения,Дата начала показа фильма,Примечание,Идентификатор записи реестра,Аннотация,Вид Фильма,Категория,...,Код,Права проката переданы,Порядковый номер переоформления/продления,Должность подписанта,ФИО подписанта,Запись удалена,Дата удаления записи,Не показывать на сайте mkrf.ru,дата создания объекта,дата обновления объекта
0,Рок’89,NaN,211004124,2024-01-30T12:00:00.000Z,NaN,NaN,10003030,NaN,Художественный,Видео,...,[],NaN,NaN,Первый заместитель Министра,С.Г. Обрывалин,NaN,NaN,NaN,2024-01-30T15:41:59+03:00,2024-02-05T14:17:27+03:00
1,Мама будет против,NaN,211004024,2024-01-30T12:00:00.000Z,2024-02-01T12:00:00.000Z,NaN,10003032,Комедия,Художественный,Видео,...,[],NaN,NaN,Первый заместитель Министра,С.Г. Обрывалин,NaN,NaN,NaN,2024-01-30T15:42:13+03:00,2024-02-05T18:23:14+03:00
2,ДЕВЯТЬ С ПОЛОВИНОЙ НЕДЕЛЬ,9 1/2 WEEKS,221005424,2024-01-30T12:00:00.000Z,2024-02-02T12:00:00.000Z,NaN,10003203,NaN,Художественный,Видео,...,[],NaN,NaN,Первый заместитель Министра,С.Г. Обрывалин,NaN,NaN,NaN,2024-01-30T16:08:51+03:00,2024-02-06T12:12:04+03:00
3,Как я встретил её маму,NaN,111001324,2024-01-30T12:00:00.000Z,NaN,NaN,10003265,NaN,Художественный,Кино,...,"[{""rightsCategoryUse"":{""code"":""01"",""name"":""Пок...",NaN,NaN,Первый заместитель Министра,С.Г. Обрывалин,NaN,NaN,NaN,2024-01-30T16:20:44+03:00,2024-03-26T14:41:20+03:00
4,Миссия в Москве,Mo Si Ke xing dong,121002924,2024-01-30T12:00:00.000Z,2024-02-08T12:00:00.000Z,NaN,10003499,NaN,Художественный,Кино,...,[],NaN,NaN,Первый заместитель Министра,С.Г. Обрывалин,NaN,NaN,NaN,2024-01-30T18:10:00+03:00,2024-01-30T18:10:00+03:00


In [4]:
pd_tools.df_info(raw_data)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103848 entries, 0 to 103847
Data columns (total 48 columns):
 #   Column                                                 Non-Null Count   Dtype  
---  ------                                                 --------------   -----  
 0   Название фильма                                        103848 non-null  object 
 1   Hаименование на иностранном языке                      51894 non-null   object 
 2   Номер удостоверения                                    103848 non-null  object 
 3   Дата регистрации удостоверения                         103846 non-null  object 
 4   Дата начала показа фильма                              99996 non-null   object 
 5   Примечание                                             29239 non-null   object 
 6   Идентификатор записи реестра                           103848 non-null  int64  
 7   Аннотация                                              92858 non-null   object 
 8   Вид Фильма                        

'First 5 rows in df'

,Название фильма,Hаименование на иностранном языке,Номер удостоверения,Дата регистрации удостоверения,Дата начала показа фильма,Примечание,Идентификатор записи реестра,Аннотация,Вид Фильма,Категория,...,Код,Права проката переданы,Порядковый номер переоформления/продления,Должность подписанта,ФИО подписанта,Запись удалена,Дата удаления записи,Не показывать на сайте mkrf.ru,дата создания объекта,дата обновления объекта
0,Рок’89,NaN,211004124,2024-01-30T12:00:00.000Z,NaN,NaN,10003030,NaN,Художественный,Видео,...,[],NaN,NaN,Первый заместитель Министра,С.Г. Обрывалин,NaN,NaN,NaN,2024-01-30T15:41:59+03:00,2024-02-05T14:17:27+03:00
1,Мама будет против,NaN,211004024,2024-01-30T12:00:00.000Z,2024-02-01T12:00:00.000Z,NaN,10003032,Комедия,Художественный,Видео,...,[],NaN,NaN,Первый заместитель Министра,С.Г. Обрывалин,NaN,NaN,NaN,2024-01-30T15:42:13+03:00,2024-02-05T18:23:14+03:00
2,ДЕВЯТЬ С ПОЛОВИНОЙ НЕДЕЛЬ,9 1/2 WEEKS,221005424,2024-01-30T12:00:00.000Z,2024-02-02T12:00:00.000Z,NaN,10003203,NaN,Художественный,Видео,...,[],NaN,NaN,Первый заместитель Министра,С.Г. Обрывалин,NaN,NaN,NaN,2024-01-30T16:08:51+03:00,2024-02-06T12:12:04+03:00
3,Как я встретил её маму,NaN,111001324,2024-01-30T12:00:00.000Z,NaN,NaN,10003265,NaN,Художественный,Кино,...,"[{""rightsCategoryUse"":{""code"":""01"",""name"":""Пок...",NaN,NaN,Первый заместитель Министра,С.Г. Обрывалин,NaN,NaN,NaN,2024-01-30T16:20:44+03:00,2024-03-26T14:41:20+03:00
4,Миссия в Москве,Mo Si Ke xing dong,121002924,2024-01-30T12:00:00.000Z,2024-02-08T12:00:00.000Z,NaN,10003499,NaN,Художественный,Кино,...,[],NaN,NaN,Первый заместитель Министра,С.Г. Обрывалин,NaN,NaN,NaN,2024-01-30T18:10:00+03:00,2024-01-30T18:10:00+03:00


,duplicates_full
duplicates_full,15


,Название фильма,Hаименование на иностранном языке,Номер удостоверения,Дата регистрации удостоверения,Дата начала показа фильма,Примечание,Идентификатор записи реестра,Аннотация,Вид Фильма,Категория,...,Код,Права проката переданы,Порядковый номер переоформления/продления,Должность подписанта,ФИО подписанта,Запись удалена,Дата удаления записи,Не показывать на сайте mkrf.ru,дата создания объекта,дата обновления объекта
duplicates_by_cols,17600,17608,98,97112,92605,22123,15,39943,103828,103843,...,102400,145,10,19981,19968,0,0,0,84229,80373


...

""
nones


""
NA placeholder


""
null placeholder


""
N/A placeholder


,<class 'str'>,<class 'int'>,<class 'float'>
Номер удостоверения,65536,38312.0,NaN
Количество серий,5302,NaN,41082.0
Метраж (для фильмов на кинопленке),4840,NaN,7845.0
Количество частей/рулонов (для фильмов на кинопленке),14150,NaN,132.0
"Продолжительность демонстрации, минуты",5537,NaN,95788.0
Возрастная категория.1,87462,16384.0,NaN


## Rename columns

In [5]:
# Define column renaming dictionary
rename_map = {
    'Название фильма': 'title',
    'Hаименование на иностранном языке': 'foreign_title',
    'Номер удостоверения': 'certificate_number',
    'Дата регистрации удостоверения': 'certificate_registration_date',
    'Дата начала показа фильма': 'screening_start_date',
    'Примечание': 'note',
    'Идентификатор записи реестра': 'registry_id',
    'Аннотация': 'synopsis',
    'Вид Фильма': 'film_type',
    'Категория': 'category',
    'Язык оригинала': 'original_language',
    'Формат кадра': 'frame_format',
    'Количество серий': 'number_of_episodes',
    'Метраж (для фильмов на кинопленке)': 'film_length',
    'Количество частей/рулонов (для фильмов на кинопленке)': 'number_of_reels',
    'Цвет': 'color',
    'Продолжительность демонстрации, часы': 'duration_hours',
    'Продолжительность демонстрации, минуты': 'duration_minutes',
    'Формат носителя': 'media_format',
    'Объем носителя': 'media_volume',
    'Единица измерения': 'volume_unit',
    'Студия-производитель': 'production_studio',
    'Страна производства': 'production_country',
    'Год производства': 'production_year',
    'Режиссер': 'director',
    'Художник': 'art_director',
    'Сценарист': 'screenwriter',
    'Оператор': 'cinematographer',
    'Композитор': 'composer',
    'Продюсер': 'producer',
    'Права на использование фильма': 'usage_rights',
    'Категория прав на использование фильма': 'rights_category',
    'Срок действия прав на использование фильма': 'rights_expiry_date',
    'Срок действия прав на способ использования фильма': 'usage_method_expiry_date',
    'Возрастная категория': 'age_rating',
    'Возрастная категория.1': 'age_rating_2',
    'Дубляж (где дублирован)': 'dubbing_location',
    'Срок действия прав проката до': 'distribution_rights_expiry',
    'Код': 'code',
    'Права проката переданы': 'rights_transferred',
    'Порядковый номер переоформления/продления': 'reissue_number',
    'Должность подписанта': 'signer_position',
    'ФИО подписанта': 'signer_name',
    'Запись удалена': 'record_deleted',
    'Дата удаления записи': 'deletion_date',
    'Не показывать на сайте mkrf.ru': 'hide_on_site',
    'дата создания объекта': 'created_at',
    'дата обновления объекта': 'updated_at'
}

# change keys for values and values for keys for using in ddl
cols_descriptions = {v: k for k, v in rename_map.items()}

# save descriptions to file
pd.DataFrame.from_dict(cols_descriptions, orient='index', columns=['col_description']).to_csv(f'{data_path}/column_descriptions.csv', encoding='utf-8')

# Apply renaming
new_data = raw_data.rename(columns=rename_map)
# new_data.to_parquet(f'{data_path}/data-9-structure-4.parquet', index=False)
new_data.head()

,title,foreign_title,certificate_number,certificate_registration_date,screening_start_date,note,registry_id,synopsis,film_type,category,...,code,rights_transferred,reissue_number,signer_position,signer_name,record_deleted,deletion_date,hide_on_site,created_at,updated_at
0,Рок’89,NaN,211004124,2024-01-30T12:00:00.000Z,NaN,NaN,10003030,NaN,Художественный,Видео,...,[],NaN,NaN,Первый заместитель Министра,С.Г. Обрывалин,NaN,NaN,NaN,2024-01-30T15:41:59+03:00,2024-02-05T14:17:27+03:00
1,Мама будет против,NaN,211004024,2024-01-30T12:00:00.000Z,2024-02-01T12:00:00.000Z,NaN,10003032,Комедия,Художественный,Видео,...,[],NaN,NaN,Первый заместитель Министра,С.Г. Обрывалин,NaN,NaN,NaN,2024-01-30T15:42:13+03:00,2024-02-05T18:23:14+03:00
2,ДЕВЯТЬ С ПОЛОВИНОЙ НЕДЕЛЬ,9 1/2 WEEKS,221005424,2024-01-30T12:00:00.000Z,2024-02-02T12:00:00.000Z,NaN,10003203,NaN,Художественный,Видео,...,[],NaN,NaN,Первый заместитель Министра,С.Г. Обрывалин,NaN,NaN,NaN,2024-01-30T16:08:51+03:00,2024-02-06T12:12:04+03:00
3,Как я встретил её маму,NaN,111001324,2024-01-30T12:00:00.000Z,NaN,NaN,10003265,NaN,Художественный,Кино,...,"[{""rightsCategoryUse"":{""code"":""01"",""name"":""Пок...",NaN,NaN,Первый заместитель Министра,С.Г. Обрывалин,NaN,NaN,NaN,2024-01-30T16:20:44+03:00,2024-03-26T14:41:20+03:00
4,Миссия в Москве,Mo Si Ke xing dong,121002924,2024-01-30T12:00:00.000Z,2024-02-08T12:00:00.000Z,NaN,10003499,NaN,Художественный,Кино,...,[],NaN,NaN,Первый заместитель Министра,С.Г. Обрывалин,NaN,NaN,NaN,2024-01-30T18:10:00+03:00,2024-01-30T18:10:00+03:00


## Data transformation

In [45]:
# count values with non-digit characters in 'certificate_number'
len(new_data['certificate_number']) - new_data['certificate_number'].str.isdigit().sum()

40064

In [ ]:
foreign_title

In [60]:
def not_same_types(df: pd.DataFrame, col_name: str) -> pd.Series:
    """
    Show dtypes in column col_name if they are not the same.
    If all values have the same type, return an empty Series.
    """
    col_types = df[col_name].apply(type).value_counts()
    if len(col_types) > 1:
        return col_types   

In [66]:
not_same = {}
for col in new_data.columns:
    if not_same_types(new_data, col) is not None:
        not_same[col] = not_same_types(new_data, col)

In [68]:
pd.DataFrame.from_dict(not_same, orient='index')

,<class 'float'>,<class 'str'>,<class 'int'>
foreign_title,51954,51894,NaN
certificate_registration_date,2,103846,NaN
screening_start_date,3852,99996,NaN
note,74609,29239,NaN
synopsis,10990,92858,NaN
film_type,5,103843,NaN
frame_format,89606,14242,NaN
number_of_episodes,98546,5302,NaN
film_length,99008,4840,NaN
number_of_reels,89698,14150,NaN


In [57]:
# find dtypes in column 'certificate_number'
not_same_types(new_data, 'certificate_number')

In [47]:
# find spaces in 'certificate_number' and replace them with empty string
new_data['certificate_number'] = new_data['certificate_number'].str.replace(' ', '')
new_data.query('certificate_number.str.isdigit() == False')['certificate_number'].sort_values


<bound method Series.sort_values of 12346           нет
21273    121012208А
22050    221113209_
29656    2210-45210
31425     213-11610
            ...    
88468    221123801Д
88489    221008000Д
88564    221050601Д
88565    221098103Д
88566    221104600Д
Name: certificate_number, Length: 1752, dtype: object>

In [48]:
# convert 'certificate_number' to string
new_data['certificate_number'] = new_data['certificate_number'].astype(str)


In [49]:
# show dtype of 'certificate_number'
new_data['certificate_number'].dtype

dtype('O')

In [52]:
new_data['number_of_episodes'].apply(type).value_counts()

number_of_episodes
<class 'float'>    98546
<class 'str'>       5302
Name: count, dtype: int64

In [50]:
new_data.to_parquet(f'{data_path}/data-9-structure-4.parquet', index=False)

ArrowInvalid: ("Could not convert '1' with type str: tried to convert to double", 'Conversion failed for column number_of_episodes with type object')

In [6]:
# Создание курсора
conn = conn_pg()

cur = conn.cursor()

# Создание таблицы
cur.execute('''
CREATE TABLE IF NOT EXISTS employees (
    id SERIAL PRIMARY KEY,
    name VARCHAR(100),
    position VARCHAR(100),
    salary NUMERIC
);
''')
print("Table 'employees' created")

# Вставка данных в таблицу
cur.execute('''
INSERT INTO employees (name, position, salary)
VALUES ('Alice', 'Manager', 70000),
        ('Bob', 'Developer', 60000),
        ('Charlie', 'Designer', 50000);
''')
print("Data inserted into 'employees' table")

# Сохранение изменений
conn.commit()


Table 'employees' created
Data inserted into 'employees' table


In [7]:
# Закрытие курсора и соединения
conn_pg_close(conn)

In [9]:
try:
    # Создание таблицы
    client.command('''
    CREATE TABLE IF NOT EXISTS dbch.employees (
        id UInt32,
        name String,
        position String,
        salary Float32
    ) ENGINE = MergeTree()
    ORDER BY id;
    ''')
    print("Table 'employees' created in ClickHouse")

except Exception as e:
    print(f"Error connecting to ClickHouse: {e}")


Table 'employees' created in ClickHouse


In [15]:
client.query('select * from dbch.employees')

In [16]:
# # Correct way to insert rows
# data = [
#     (1, 'Alice', 'Manager', 70000.0),
#     (2, 'Bob', 'Developer', 60000.0),
#     (3, 'Charlie', 'Designer', 50000.0)
# ]

# client.insert('dbch.employees', data, column_names=['id', 'name', 'position', 'salary'])